In [71]:
import sympy as sy
from sympy.utilities.codegen import codegen

import os
from pathlib import Path


In [72]:
N = 2  #次元

x = sy.Matrix(sy.MatrixSymbol('x', N, 1))
x_dot = sy.Matrix(sy.MatrixSymbol('x_dot', N, 1))


#x_norm = x.norm(ord=2)
x_norm = 0
for i in range(N):
    x_norm += x[i, 0]**2
x_norm = sy.sqrt(x_norm)# ** (1/2)
x_hat = x / x_norm

x_norm

sqrt(x[0, 0]**2 + x[1, 0]**2)

In [73]:
x

Matrix([
[x[0, 0]],
[x[1, 0]]])

In [74]:
### 慣性行列 ###

sigma_alpha, sigma_gamma, w_u, w_l, alpha, epsilon = sy.symbols('sigma_alpha, sigma_gamma, w_u, w_l, alpha, epsilon')


alpha_x = sy.exp(-((x_norm**2) / (2 * sigma_alpha**2)))
gamma_x = sy.exp(-((x_norm**2) / (2 * sigma_gamma**2)))
w_x = gamma_x * w_u + (1 - gamma_x) * w_l
nabla_x_phi = (1 - sy.exp(-2 * alpha * x_norm)) / (1 + sy.exp(-2 * alpha * x_norm)) * x_hat


M = w_x  * ((1 - alpha_x) * nabla_x_phi * nabla_x_phi.T + (alpha_x + epsilon) * sy.eye(N))

M = sy.simplify(M)
M

Matrix([
[(w_l*(1 - exp(-(x[0, 0]**2 + x[1, 0]**2)/(2*sigma_gamma**2))) + w_u*exp(-(x[0, 0]**2 + x[1, 0]**2)/(2*sigma_gamma**2)))*((1 - exp(2*alpha*sqrt(x[0, 0]**2 + x[1, 0]**2)))**2*(1 - exp(-(x[0, 0]**2 + x[1, 0]**2)/(2*sigma_alpha**2)))*x[0, 0]**2 + (epsilon + exp(-(x[0, 0]**2 + x[1, 0]**2)/(2*sigma_alpha**2)))*(exp(2*alpha*sqrt(x[0, 0]**2 + x[1, 0]**2)) + 1)**2*(x[0, 0]**2 + x[1, 0]**2))/((exp(2*alpha*sqrt(x[0, 0]**2 + x[1, 0]**2)) + 1)**2*(x[0, 0]**2 + x[1, 0]**2)),                                                                                                                                                                                                          (1 - exp(-(x[0, 0]**2 + x[1, 0]**2)/(2*sigma_alpha**2)))*(w_l*(1 - exp(-(x[0, 0]**2 + x[1, 0]**2)/(2*sigma_gamma**2))) + w_u*exp(-(x[0, 0]**2 + x[1, 0]**2)/(2*sigma_gamma**2)))*tanh(alpha*sqrt(x[0, 0]**2 + x[1, 0]**2))**2*x[0, 0]*x[1, 0]/(x[0, 0]**2 + x[1, 0]**2)],
[                                                        

In [75]:
# 曲率校xi_Mの前半を計算
partialx_m_dx = []
for i in range(N):
    print("i = ", i)
    partialx_m_dx.append(M[:, i:i+1].jacobian(x) * x_dot)
# partialx_m1 = M[:, 0:1].jacobian(x)
# partialx_m2 = M[:, 1:2].jacobian(x)
# partialx_m3 = M[:, 2:3].jacobian(x)

# print(sy.simplify(partialx_m1))


xi_M_before_ = partialx_m_dx[0]
for i in range(1, N):
    print("i = ", i)
    xi_M_before_ = xi_M_before_.row_join(partialx_m_dx[i])
# xi_M_before_before = partialx_m1_dx.row_join(partialx_m2_dx)
# xi_M_before_before = xi_M_before_before.row_join(partialx_m3_dx)

#xi_M_before_ = sy.simplify(xi_M_before_)
print(xi_M_before_.shape)

xi_M_before = xi_M_before_ * x_dot

# #print(xi_M_before)

# # 曲率校xi_Mの後半を計算
xi_M_after_ = x_dot.T * M * x_dot
xi_M_after = -1/2* (xi_M_after_.jacobian(x)).T #　sympyにはナブラあるかも？

# # 合体
xi_M = xi_M_before + xi_M_after
xi_M = sy.simplify(xi_M)  # Elisじゃ無理
# print(xi_M)

i =  0
i =  1
i =  1
(2, 2)


In [77]:
def gen(func, name):
    [(c_name, c_code), (h_name, c_header)] = codegen(
        name_expr=(name, func),
        language="C",
        project= name + "project",
        to_files=False
    )

    f = open(c_name, 'w')
    f.write(c_code)
    f.close()

    f = open(h_name, 'w')
    f.write(c_header)
    f.close()


gen(xi_M, "rmp2_attractor_xi")